In [34]:
from bs4 import BeautifulSoup as bs
import requests
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)

C:\Users\Chloe\AppData\Local\Temp\ipykernel_4300\3936863406.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)


In [38]:
# creates a datafram with all of the needed columns including "title", "author", etc.
df = pd.DataFrame({'title': pd.Series(dtype='str'),
                   'director': pd.Series(dtype='str'),
                   'release_year': pd.Series(dtype='str'),
                   'length': pd.Series(dtype='str'),
                   'genre': pd.Series(dtype='str'),
                   'imdb_score': pd.Series(dtype='str'),
                   'rating' : pd.Series(dtype='str'),
                   'gross_US_CA' : pd.Series(dtype='str'),
})

print(df)

#specifies the website to scrape and use selenium
driver.get("https://www.imdb.com/search/title/?title_type=feature&sort=num_votes,desc")


i = 0
while True:
    try:
        # waits until the head-block class is displayed
        WebDriverWait(driver, 1).until(
            lambda s: s.find_element(By.CLASS_NAME, 'article').is_displayed()
        )
    except TimeoutException:
        break

    url = driver.current_url
    response = requests.get(url)
    html = response.content
    #creates of beautiful soup object of the current url
    soup = bs(html, "lxml")
    
    time.sleep(2)
    #makes a list of each div with a class of "item-info"
    all_item_info = soup.find_all("div", class_="lister-item-content")
    #for each item in the list, gather all the relevant data about the books
    for item_info in all_item_info:
        try:
            movie_title = item_info.find("h3", class_="lister-item-header").find("a").get_text(strip=True)
        except:
            movie_title = pd.NA
        try:
            movie_director = item_info.find("p", class_="").find("a").get_text(strip=True)
        except:
            movie_director = pd.NA
        try:
            movie_release = item_info.find("h3", class_="lister-item-header").find("span", class_="lister-item-year text-muted unbold").get_text(strip=True)
        except:
            movie_release = pd.NA
        try:
            movie_length = item_info.find("p", class_="text-muted").find("span", class_="runtime").get_text(strip=True)
        except:
            movie_length = pd.NA
        try:
            movie_genre = item_info.find("p", class_="text-muted").find("span", class_="genre").get_text(strip=True)
        except:
            movie_genre = pd.NA
        try:
            movie_score = item_info.find("div", class_="ratings-bar").find("div", class_="inline-block ratings-imdb-rating").find("strong").get_text(strip=True)
        except:
            movie_score = pd.NA
        try:
            movie_rating = item_info.find("p", class_="text-muted").find("span", class_="certificate").get_text(strip=True)
        except:
            movie_rating = pd.NA
        try:
            movie_gross = item_info.find("p", class_="sort-num_votes-visible").find_all("span")[-1].get_text(strip=True)
        except:
            movie_gross = pd.NA
        
        #since these values are sometimes non-existent, I use a try-except block
        #to set the value to NaN if there is an error (meaning the value doesn't exist)

        
        #adds each of these values as a new row to the dataframe
        df = pd.concat([df,pd.Series({'title': movie_title,
                       'director': movie_director,
                       'release_year': movie_release,
                       'length': movie_length,
                       'genre' : movie_genre,
                       'imdb_score' : movie_score,
                       'rating' : movie_rating,
                       'gross_US_CA' : movie_gross,

                                      
                      }).to_frame().T], ignore_index=True)

    #makes sure the next button exists before trying to click it
    try:
        load_more = driver.find_element(By.LINK_TEXT,'Next »')
        if load_more: driver.execute_script("arguments[0].click();", load_more)
    except: pass
                       

    #after each page it increments i by 1, once i is greater than 33 we have reached
    #the last page so it breaks out of the while statement
    i += 1
    if i > 20:
        break

#saves the dataframe as a csv file
df.to_csv('imdb_movies.csv')
df

Empty DataFrame
Columns: [title, director, release_year, length, genre, imdb_score, rating, gross_US_CA]
Index: []


,title,director,release_year,length,genre,imdb_score,rating,gross_US_CA
0,The Shawshank Redemption,Frank Darabont,(1994),142 min,Drama,9.3,R,$28.34M
1,The Dark Knight,Christopher Nolan,(2008),152 min,"Action, Crime, Drama",9.0,PG-13,$534.86M
2,Inception,Christopher Nolan,(2010),148 min,"Action, Adventure, Sci-Fi",8.8,PG-13,$292.58M
3,Fight Club,David Fincher,(1999),139 min,Drama,8.8,R,$37.03M
4,Forrest Gump,Robert Zemeckis,(1994),142 min,"Drama, Romance",8.8,PG-13,$330.25M
...,...,...,...,...,...,...,...,...
1045,Love & Other Drugs,Edward Zwick,(2010),112 min,"Comedy, Drama, Romance",6.7,R,$32.37M
1046,Enchanted,Kevin Lima,(2007),107 min,"Animation, Adventure, Comedy",7.1,PG,$127.81M
1047,The Raid: Redemption,Gareth Evans,(2011),101 min,"Action, Crime, Thriller",7.6,R,$4.11M
1048,"South Park: Bigger, Longer & Uncut",Trey Parker,(1999),81 min,"Animation, Comedy, Fantasy",7.7,R,$52.04M
